In [7]:
import numpy as np

covs_all = np.load('../data/simulation2/covs_all.npy')
cov = np.load('../data/simulation2/cov_pop.npy')

In [8]:
num_cells = 10
len_series = np.array([68, 116, 216, 216,  94, 216, 216, 190, 216, 216],dtype=np.int64)

In [9]:
ns02 = np.zeros((21,11),dtype=np.int64)

In [10]:
list0 = [0,3,4,6,7,9,10,11,12,13,15,18,19,20]
list1 = [2,7,8,9,12,14,15,16,17,18,20,24,25,26,28,31,32,33]
list2 = [2,4,5,6,7,8,9,10]

In [11]:
for i in list0:
    for j in list2:
        print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+4,j+59],[i+4,j+59])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+4,j+59],[i+4,j+59])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns02[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns02[i,j])

0 2
0 2 48400
0 4
0 4 1245
0 5
0 5 18490
0 6
0 6 3112
0 7
0 7 2855
0 8
0 8 1078
0 9
0 9 5006
0 10
0 10 2136
3 2
3 2 57812
3 4
3 4 1951
3 5
3 5 582383
3 6
3 6 11686
3 7
3 7 5081
3 8
3 8 2062
3 9
3 9 640434
3 10
3 10 7163
4 2
4 2 0
4 4
4 4 0
4 5
4 5 919
4 6
4 6 0
4 7
4 7 163
4 8
4 8 0
4 9
4 9 31
4 10
4 10 0
6 2
6 2 1
6 4
6 4 42181
6 5
6 5 60226
6 6
6 6 11720
6 7
6 7 3245512
6 8
6 8 3434
6 9
6 9 4879
6 10
6 10 135
7 2
7 2 6235
7 4
7 4 627198
7 5
7 5 9456
7 6
7 6 51646
7 7
7 7 690994
7 8
7 8 539365
7 9
7 9 4509
7 10
7 10 23658
9 2
9 2 9
9 4
9 4 74
9 5
9 5 5382
9 6
9 6 983
9 7
9 7 751495
9 8
9 8 1
9 9
9 9 22
9 10
9 10 0
10 2
10 2 0
10 4
10 4 11
10 5
10 5 384
10 6
10 6 0
10 7
10 7 198
10 8
10 8 0
10 9
10 9 0
10 10
10 10 0
11 2
11 2 26
11 4
11 4 0
11 5
11 5 6
11 6
11 6 0
11 7
11 7 206
11 8
11 8 0
11 9
11 9 0
11 10
11 10 0
12 2
12 2 382
12 4
12 4 858
12 5
12 5 72932
12 6
12 6 10999
12 7
12 7 4789385
12 8
12 8 144
12 9
12 9 13265
12 10
12 10 57
13 2
13 2 3
13 4
13 4 13933
13 5
13 5 643266
13 6


In [12]:
with open('../data/simulation2/ns02.npy','wb') as f:
    np.save(f, ns02)